### Speech Processing Analysis & Machine Learning & Ensemble Modelling




In [ ]:
!pip install spafe
!pip install pip install scikit-plot

     |████████████████████████████████| 77 kB 3.2 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
for i in os.listdir("/content/drive/MyDrive/archive (1)/augmented_dataset/"):
    print(str(len(os.listdir("/content/drive/MyDrive/archive (1)/augmented_dataset/"+i))) +" "+ i +" audios")

1356 bed audios
1357 bird audios
1424 cat audios
1198 down audios
1498 dog audios
1133 eight audios
1092 five audios
960 go audios
2400 four audios
1481 happy audios
1505 left audios
2392 house audios
1253 marvel audios
1144 nine audios
1002 no audios
2244 off audios
2228 on audios
1276 one audios
1296 right audios
1411 seven audios
1485 six audios
1463 sheila audios
1188 three audios
1485 stop audios
1188 tree audios
902 two audios
1187 up audios
957 wow audios
1244 yes audios
1306 zero audios


In [ ]:
import os
for i in os.listdir("/content/drive/MyDrive/archive (1)/augmented_dataset_verynoisy/"):
    print(str(len(os.listdir("/content/drive/MyDrive/archive (1)/augmented_dataset_verynoisy/"+i))) +" "+ i +" audios")

1489 dog audios
1402 cat audios
1113 eight audios
1223 down audios
1112 five audios
960 go audios
2400 four audios
1481 happy audios
2392 house audios
1485 left audios
1253 marvel audios
1145 nine audios
963 no audios
2252 off audios
2228 on audios
1276 one audios
1276 right audios
1411 seven audios
1463 sheila audios
1500 six audios
1485 stop audios
2028 three audios
2072 tree audios
902 two audios
1187 up audios
957 wow audios
1547 yes audios
1602 zero audios
1367 bed audios
1366 bird audios


In [ ]:
import os
for i in os.listdir("/content/drive/MyDrive/speech_commands_v0.01/"):
    print(str(len(os.listdir("/content/drive/MyDrive/speech_commands_v0.01/"+i))) +" "+ i +" audios")

2357 five audios
2352 eight audios
2372 four audios
2372 go audios
1742 happy audios
1750 house audios
2353 left audios
1746 marvin audios
2365 nine audios
1881 no audios
2357 off audios
2392 on audios
2370 one audios
2367 right audios
2392 seven audios
1734 sheila audios
2387 six audios
2390 stop audios
2346 three audios
1733 tree audios
2373 two audios
1745 wow audios
2387 yes audios
2376 up audios
2376 zero audios
1713 bed audios
1731 bird audios
1733 cat audios
2359 down audios
1746 dog audios


In [ ]:
# 1D cnn for SER

from keras.models import Model, Sequential
from keras import optimizers
from keras.layers import Input, Conv1D, BatchNormalization, MaxPooling1D, LSTM, Dense, Activation, Layer
from emodata1d import load_data
from keras.utils import to_categorical
import keras.backend as K
import argparse
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras_self_attention import SeqSelfAttention

def emo1d(input_shape, num_classes, args):
    model = Sequential(name='Emo1D')

    # LFLB1
    model.add(Conv1D(filters=64, kernel_size=(3), strides=1, padding='same', data_format='channels_last',
                     input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(MaxPooling1D(pool_size=4, strides=4))

    # LFLB2
    model.add(Conv1D(filters=64, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(MaxPooling1D(pool_size=4, strides=4))

    # LFLB3
    model.add(Conv1D(filters=128, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(MaxPooling1D(pool_size=4, strides=4))

    # LFLB4
    model.add(Conv1D(filters=128, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(MaxPooling1D(pool_size=4, strides=4))

    # LSTM
    model.add(LSTM(units=args.num_fc,return_sequences=True))
    model.add(SeqSelfAttention(attention_activation='tanh'))
    model.add(LSTM(units=args.num_fc,return_sequences=False))

    # FC
    model.add(Dense(units=num_classes, activation='softmax'))

    # Model compilation
    opt = optimizers.SGD(lr=args.learning_rate, decay=args.decay, momentum=args.momentum, nesterov=True)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    return model


def train(model, x_tr, y_tr, x_val, y_val, args):
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=8)
    mc = ModelCheckpoint('best_model.h5', monitor='val_categorical_accuracy', mode='max', verbose=1,
                         save_best_only=True)
    history = model.fit(x_tr, y_tr, epochs=args.num_epochs, batch_size=args.batch_size, validation_data=(x_val, y_val),
                        callbacks=[es, mc])
    return model


def test(model, x_t, y_t):
    saved_model = load_model('best_model.h5',custom_objects={'SeqSelfAttention':SeqSelfAttention})
    score = saved_model.evaluate(x_t, y_t, batch_size=20)
    print(score)
    return score


def loadData(features,target,labels):
  # converting labels into numeric
    le = preprocessing.LabelEncoder()
    target=le.fit_transform(target)
    # features = preprocessing.MinMaxScaler().fit_transform(features)
    target=np_utils.to_categorical(target, num_classes=len(labels))
    features = np.array(features).reshape(-1,features.shape[1],1)
    x_tr, x_val, y_tr, y_val = train_test_split(np.array(features),np.array(target),stratify=y,test_size = 0.2,random_state=777,shuffle=True)
    x_t=x_val
    y_t=y_val
    #x_tr, y_tr, x_t, y_t, x_val, y_val = load_data()

    #x_tr = x_tr.reshape(-1, x_tr.shape[1], 1)
    #x_t = x_t.reshape(-1, x_t.shape[1], 1)
    #x_val = x_val.reshape(-1, x_val.shape[1], 1)
    #y_tr = to_categorical(y_tr)
    #y_t = to_categorical(y_t)
    #y_val = to_categorical(y_val)
    return x_tr, y_tr, x_t, y_t, x_val, y_val

def dataspit(feature,target):
  # converting labels into numeric
  le = preprocessing.LabelEncoder()
  target=le.fit_transform(target)
  # features = preprocessing.MinMaxScaler().fit_transform(features)
  feature_train, feature_test, target_train, target_test = train_test_split(feature,target)

def Data_preparation(train_path):
  warnings.filterwarnings('ignore')
  features_cal = feature_calculation()
  # setting the path where all file's folder are
  root = train_path
  Features_data = pd.DataFrame(columns=['features','class'])
  MFCC_features=pd.DataFrame(columns=['features','class'])
  MSS_features=pd.DataFrame(columns=['features','class'])
  poly_features=pd.DataFrame(columns=['features','class'])
  ZCR_features=pd.DataFrame(columns=['features','class'])

  MFCCMSS_features=pd.DataFrame(columns=['features','class'])
  MFCCpoly_features=pd.DataFrame(columns=['features','class'])
  MFCCZCR_features=pd.DataFrame(columns=['features','class'])

  MSSpoly_features=pd.DataFrame(columns=['features','class'])
  MSSZCR_features=pd.DataFrame(columns=['features','class'])
  polyZCR_features=pd.DataFrame(columns=['features','class'])

  MFCCMSSpoly_features=pd.DataFrame(columns=['features','class'])
  MFCCpolyZCR_features=pd.DataFrame(columns=['features','class'])
  MSSpolyZCR_features=pd.DataFrame(columns=['features','class'])
  all_wave = pd.DataFrame(columns=['features','class'])
  all_label = []
  i = 0
  sample_rate = 16000
  no_of_samples = 300
  MainFolder=train_path
  labels=os.listdir(MainFolder)
  # Loading the features in the dataframe
  for label in labels:
    print(label)
    folders = os.path.join(root,label)
    items = os.listdir(folders)
    for item in items:
      path = os.path.join(folders,item)
      #Convert .wave into array
      samples, sample_rate=librosa.load(path ,sr=sample_rate)
      #Extract Feautures
      #samples=self.speech_preprocessing( samples, sample_rate)
      MFCC = features_cal.mfcc_feature(samples , sample_rate)
      #RPLP=features_cal.RPLP_feature(samples , sample_rate)
      MSS = features_cal.melspectrogram_feature(samples , sample_rate)
      poly = features_cal.poly_feature(samples , sample_rate)
      ZCR = features_cal.zero_crossing_rate_features(samples , sample_rate)
      # flatten an array
      MFCC = MFCC.flatten()
      #RPLP=RPLP.flatten()
      MSS = MSS.flatten()
      poly = poly.flatten()
      ZCR = ZCR.flatten()
      # normalizing
      # MFCC = normalize(MFCC)
      features = np.concatenate(( MFCC,MSS, poly, ZCR))
      features1=MFCC
      features2=MSS
      features3=poly
      features4=ZCR

      features_combo1 = np.concatenate(( MFCC,MSS))
      features_combo2 = np.concatenate(( MFCC,poly))
      features_combo3 = np.concatenate(( MFCC,ZCR))
      features_combo4 = np.concatenate(( MSS, poly))
      features_combo5 = np.concatenate(( MSS,ZCR))
      features_combo6 = np.concatenate(( poly,ZCR))

      features_combo7 = np.concatenate(( MFCC,MSS, poly))
      features_combo8 = np.concatenate(( MFCC,poly, ZCR))
      features_combo9 = np.concatenate(( MSS, poly, ZCR))
      # padding and trimming
      max_len = 8000
      pad_width = max_len - features.shape[0]
      if pad_width > 0:
        features = np.pad(features, pad_width=((0, pad_width)), mode='constant')
      features = features[:max_len]
      # padding and trimming
      max_len = 8000
      pad_width = max_len - features1.shape[0]
      if pad_width > 0:
              features1 = np.pad(features1, pad_width=((0, pad_width)), mode='constant')
      features1 = features1[:max_len]
      # padding and trimming
      max_len = 8000
      pad_width = max_len - features2.shape[0]
      if pad_width > 0:
        features2 = np.pad(features2, pad_width=((0, pad_width)), mode='constant')
      features2 = features2[:max_len]
      # padding and trimming]
      max_len = 8000
      pad_width = max_len - features3.shape[0]
      if pad_width > 0:
        features3 = np.pad(features3, pad_width=((0, pad_width)), mode='constant')
      features3 = features3[:max_len]
      # padding and trimming
      max_len = 8000
      pad_width = max_len - features4.shape[0]
      if pad_width > 0:
              features4 = np.pad(features4, pad_width=((0, pad_width)), mode='constant')
      features4 = features4[:max_len]
      # padding and trimming
      max_len = 8000
      pad_width = max_len - features_combo1.shape[0]
      if pad_width > 0:
              features_combo1 = np.pad(features_combo1, pad_width=((0, pad_width)), mode='constant')
      features_combo1= features_combo1[:max_len]
      # padding and trimming
      max_len = 8000
      pad_width = max_len - features_combo2.shape[0]
      if pad_width > 0:
        features_combo2 = np.pad(features_combo2, pad_width=((0, pad_width)), mode='constant')
      features_combo2 = features_combo2[:max_len]
      # padding and trimming
      max_len = 8000
      pad_width = max_len - features_combo3.shape[0]
      if pad_width > 0:
        features_combo3 = np.pad(features_combo3, pad_width=((0, pad_width)), mode='constant')
       features_combo3 = features_combo3[:max_len]
      # padding and trimming
      max_len = 8000
      pad_width = max_len - features_combo4.shape[0]
      if pad_width > 0:
        features_combo4 = np.pad(features_combo4, pad_width=((0, pad_width)), mode='constant')
      features_combo4 = features_combo4[:max_len]
      # padding and trimming
      max_len = 8000
      pad_width = max_len - features_combo5.shape[0]
      if pad_width > 0:
        features_combo5= np.pad(features_combo5, pad_width=((0, pad_width)), mode='constant')
      features_combo5 = features_combo5[:max_len]
      # padding and trimming
      max_len = 8000
      pad_width = max_len - features_combo6.shape[0]
      if pad_width > 0:
        features_combo6 = np.pad(features_combo6, pad_width=((0, pad_width)), mode='constant')
      features_combo6 = features_combo6[:max_len]
      # padding and trimming
      max_len = 8000
      pad_width = max_len - features_combo7.shape[0]
      if pad_width > 0:
        features_combo7 = np.pad(features_combo7, pad_width=((0, pad_width)), mode='constant')
      features_combo7 = features_combo7[:max_len]
      # padding and trimming
      max_len = 8000
      pad_width = max_len - features_combo8.shape[0]
      if pad_width > 0:
        features_combo8 = np.pad(features_combo8, pad_width=((0, pad_width)), mode='constant')
      features_combo8 = features_combo8[:max_len]
      # padding and trimming
      max_len = 8000
      pad_width = max_len - features_combo9.shape[0]
      if pad_width > 0:
        features_combo9 = np.pad(features_combo9, pad_width=((0, pad_width)), mode='constant')
      features_combo9 = features_combo9[:max_len]
      if(len(samples)== 8000):
        Featured_data.loc[i] = [features, Label]
        MFCC_features.loc[i] = [features1, Label]
        MSS_features.loc[i] = [features2, Label]
        poly_features.loc[i] = [features3, Label]
        ZCR_features.loc[i] = [features4, Label]
        MFCCMSS_features.loc[i] = [features_combo1, Label]
        MFCCpoly_features.loc[i] = [features_combo2, Label]
        MFCCZCR_features.loc[i] = [features_combo3, Label]
        MSSpoly_features.loc[i] = [features_combo4, Label]
        MSSZCR_features.loc[i] = [features_combo5, Label]
        polyZCR_features.loc[i] = [features_combo6, Label]
        MFCCMSSpoly_features.loc[i] = [features_combo7, Label]
        MFCCpolyZCR_features.loc[i] = [features_combo8, Label]
        MSSpolyZCR_features.loc[i] = [features_combo9, Label]
        all_wave.loc[i] = [samples, Label]
      i += 1
  #np.set_printoptions(threshold=sys.maxsize)
  Features_data.to_csv(self.data_path+'All_Features_datanoisy.csv')
  MFCC_features.to_csv(self.data_path+'MFCC_features_datanoisy.csv')
  MSS_features.to_csv(self.data_path+'MSS_features_datanoisy.csv')
  poly_features.to_csv(self.data_path+'poly_features_datanoisy.csv')
  ZCR_features.to_csv(self.data_path+'ZCR_features_datanoisy.csv')
  MFCCMSS_features.to_csv(self.data_path+'MFCCMSS_features_datanoisy.csv')
  MFCCpoly_features.to_csv(self.data_path+'MFCCpoly_features_datanoisy.csv')
  MFCCZCR_features.to_csv(self.data_path+'MFCCZCR_features_datanoisy.csv')
  MSSpoly_feature.to_csv(self.data_path+'MSSpoly_feature_datanoisy.csv')
  MSSZCR_features.to_csv(self.data_path+'MSSZCR_features_datanoisy.csv')
  polyZCR_features.to_csv(self.data_path+'polyZCR_features_datanoisy.csv')
  MFCCMSSpoly_features.to_csv(self.data_path+'MFCCMSSpoly_features_datanoisy.csv')
  MFCCpolyZCR_features.to_csv(self.data_path+'MFCCpolyZCR_features_datanoisy.csv')
  MSSpolyZCR_features.to_csv(self.data_path+'MSSpolyZCR_features_datanoisy.csv')
  all_wave.to_csv(self.data_path+'all_wave_train_datanoisy.csv')
  feature_all=np.array(Features_data['features'].tolist())
  feature1=np.array(MFCC_features['features'].tolist())
  feature2=np.array(MSS_features['features'].tolist())
  feature3=np.array(poly_features['features'].tolist())
  feature4=np.array(ZCR_features['features'].tolist())
  feature_combo1=np.array(MFCCMSS_features['features'].tolist())
  feature_combo2=np.array(MFCCpoly_features['features'].tolist())
  feature_combo3=np.array(MFCCZCR_features['features'].tolist())
  feature_combo4=np.array(MSSpoly_feature['features'].tolist())
  feature_combo5=np.array(MSSZCR_features['features'].tolist())
  feature_combo6=np.array(polyZCR_features['features'].tolist())
  feature_combo7=np.array(MFCCMSSpoly_features['features'].tolist())
  feature_combo8=np.array(MFCCpolyZCR_features['features'].tolist())
  feature_combo9=np.array(MSSpolyZCR_features['features'].tolist())
  target = Features_data.iloc[:,-1]



bed
bird
cat
down


In [ ]:
if __name__ == "__main__":
    import numpy as np
    import matplotlib.pyplot as plt

    parser = argparse.ArgumentParser()
    args = parser.parse_args()

    # load data
    x_tr, y_tr, x_t, y_t, x_val, y_val = loadData()

    args.num_fc = 64
    args.batch_size = 32
    args.num_epochs = 1500  # best model will be saved before number of epochs reach this value
    args.learning_rate = 0.0001
    args.decay = 1e-6
    args.momentum = 0.9

    # define model
    model = emo1d(input_shape=x_tr.shape[1:], num_classes=len(np.unique(np.argmax(y_tr, 1))), args=args)
    model.summary()

    # train model
    model = train(model, x_tr, y_tr, x_val, y_val, args=args)

    # test model
    score = test(model, x_t, y_t) #[0.9742442428736396, 0.6445672231594283]